# Evaluate UMLS

In [1]:
import pandas as pd
from pathlib import Path

In [2]:
annotations = pd.read_csv(f'../annotations/combined_pd.csv')
patients = annotations[annotations.group_name == 'patients']
# Only keep those where `umls` annotation is not empty
patients = patients[patients.umls.notnull()]
fields_keep = ['pmcid', 'group_name', 'subgroup_name', 'count', 'umls', 'umls_posthoc', 'diagnosis']
patients = patients[fields_keep]
outputs = Path('../outputs')

In [3]:
annotations[annotations.umls_posthoc.notnull()]

,Unnamed: 0,group_name,subgroup_name,project_name,annotator_name,pmcid,umls,umls_posthoc,diagnosis,count,male count,age mean,female count,age minimum,age maximum,age median
8,109,patients,amci,participant_demographics,Jerome_Dockes,7838677,C5394908,C1270972,amnestic mild cognitive impairment,11,4.0,NaN,7.0,65.0,81.0,NaN
12,79,patients,ais,participant_demographics,Jerome_Dockes,8785614,C5680049,C0948008,arterial ischemic stroke,26,NaN,NaN,NaN,NaN,NaN,NaN
34,372,patients,_,participant_demographics,Jerome_Dockes,4048172,C0740418,C0150055,Chronic Back Pain,13,4.0,51.8,9.0,43.0,65.0,NaN
35,405,patients,_,participant_demographics,Jerome_Dockes,7416060,C4518790,C0024809,Chronic cannabis users,54,NaN,NaN,NaN,NaN,NaN,NaN
37,64,patients,_,participant_demographics,Jerome_Dockes,4473263,C0586325,C0038454,chronic left-hemisphere stroke,32,27.0,62.0,5.0,NaN,NaN,NaN
39,265,patients,_,participant_demographics,Jerome_Dockes,3742334,C4518790,C0024809,chronic marijuana use,10,8.0,NaN,2.0,NaN,NaN,NaN
40,346,patients,_,participant_demographics,Jerome_Dockes,6585311,C2711916,C0018989,chronic post-stroke hemiparesis,10,6.0,66.7,4.0,NaN,NaN,NaN
43,39,patients,_,participant_demographics,Jerome_Dockes,8459240,C4237066,C4763868,cocaine use disorder,55,27.0,NaN,28.0,NaN,NaN,NaN
45,341,patients,_,participant_demographics,Jerome_Dockes,5371603,C0011053,C1547302,deafness,14,3.0,25.0,11.0,18.0,33.0,NaN
48,411,patients,_,participant_demographics,Jerome_Dockes,4589842,"C2362914, C1269683, C0006012",C0344315,Depression,11,5.0,61.0,6.0,NaN,NaN,NaN


In [4]:
patients.umls_posthoc


0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
         ...   
141         NaN
142         NaN
143    C1270972
144         NaN
145         NaN
Name: umls_posthoc, Length: 144, dtype: object

In [5]:
# Add index for later
patients = patients.reset_index()

In [6]:
all_predictions = []
for pred in list(outputs.glob('*umls*')):
    md = pred.stem.split('_')
    if md[0] == 'chunked':
        chunk = True
        prompt = md[1]
        model_name = md[2]
        source = 'md'
    else:
        chunk = False
        source = md[1]
        prompt = md[2]
        model_name = md[3]
    pred_df = pd.read_csv(pred)

    pred_df['chunk'] = chunk    
    pred_df['source'] = source
    pred_df['prompt'] = prompt
    pred_df['model_name'] = model_name

    all_predictions.append(pred_df)



In [7]:
# For each PMCID, get the best match for each UMLS code, without replacement (i.e. only one match per UMLS code)
# If no match is found, then the UMLS code is not matched

def match_umls_codes(umls_predicitons, annot_patients):
    annot_patients = annot_patients.copy()


    for pmcid, group in annot_patients.groupby('pmcid'):
        doc_preds = umls_predicitons[umls_predicitons.pmcid == pmcid]

        if len(doc_preds) == 0:
            continue

        used_group_ixs = []
        # Iterate over annotations for each document
        for row in group.itertuples():

            # Skip if no UMLS codes
            if pd.isnull(row.umls):
                continue

            correct = [c.strip() for c in row.umls.split(', ')]
            if pd.isnull(row.umls_posthoc):
                posthoc_correct = []
            else:
                # Compute "posthoc" matches (i.e. UMLS codes added after the initial annotation)
                posthoc_correct = [c.strip() for c in row.umls_posthoc.split(', ')]
            
            # Only match if the predition group has not been matched yet
            matches = doc_preds[(doc_preds['umls_cui'].isin(correct)) & (doc_preds['group_ix'].isin(used_group_ixs) == False)]
            is_posthoc = False
            if len(matches) == 0:
                # If no match, try with posthoc
                matches = doc_preds[(doc_preds['umls_cui'].isin(posthoc_correct)) & (doc_preds['group_ix'].isin(used_group_ixs) == False)]
                is_posthoc = True

    
            if len(matches) > 0:
                # Take the best match
                m =  matches[matches['umls_prob'] == matches['umls_prob'].max()].iloc[0]
                # Add the group index to the used list
                used_group_ixs.append(m['group_ix'])
                
                annot_patients.at[row.Index, 'umls_match'] = 1
                annot_patients.at[row.Index, 'umls_posthoc_match'] = is_posthoc

                annot_patients.at[row.Index, 'umls_prob'] = m['umls_prob']
                annot_patients.at[row.Index, 'diagnosis_pred'] = m['diagnosis']
            else:
                annot_patients.at[row.Index, 'umls_match'] = 0


    annot_patients['source'] = umls_predicitons['source'].iloc[0]
    annot_patients['prompt'] = umls_predicitons['prompt'].iloc[0]
    annot_patients['model_name'] = umls_predicitons['model_name'].iloc[0]
    annot_patients['chunk'] = umls_predicitons['chunk'].iloc[0]

    return annot_patients



In [31]:
# For each prediction file, get the best match for each UMLS code, without replacement (i.e. only one match per UMLS code)
matches = []
for pred_df in all_predictions:
    matches.append(match_umls_codes(pred_df, patients))

matches = pd.concat(matches)

# Make new column for a priori matches
matches['umls_match_apriori'] = matches['umls_match']
matches.loc[(matches.umls_match == 1) & (matches.umls_posthoc_match == True), 'umls_match_apriori'] = 0

summary = matches.groupby(['source', 'prompt', 'model_name', 'chunk']).agg(
    match_n=('umls_match','sum'), 
    match_mean_accuracy= ('umls_match','mean'),
    match_n_aprior=('umls_match_apriori','sum'),
    match_mean_accuracy_aprior=('umls_match_apriori','mean'),
    mean_umls_prob=('umls_prob','mean')).reset_index()
    


In [32]:
summary.sort_values(['chunk', 'source'])

,source,prompt,model_name,chunk,match_n,match_mean_accuracy,match_n_aprior,match_mean_accuracy_aprior,mean_umls_prob
0,html,demographics-zeroshot,gpt-4o-mini-2024-07-18,False,54.0,0.782609,44.0,0.637681,0.926004
2,md,demographics-zeroshot,gpt-4o-2024-05-13,False,115.0,0.804196,96.0,0.671329,0.914526
4,md,demographics-zeroshot,gpt-4o-mini-2024-07-18,False,118.0,0.836879,97.0,0.687943,0.914584
6,md,demographics-zeroshot-ftstrict,gpt-4o-2024-05-13,False,113.0,0.795775,96.0,0.676056,0.906655
7,md,demographics-zeroshot-ftstrict,gpt-4o-mini-2024-07-18,False,116.0,0.822695,98.0,0.695035,0.864620
1,md,demographics-fewshot,gpt-4o-2024-05-13,True,100.0,0.869565,86.0,0.747826,0.940373
3,md,demographics-zeroshot,gpt-4o-2024-05-13,True,104.0,0.873950,87.0,0.731092,0.934027
5,md,demographics-zeroshot,gpt-4o-mini-2024-07-18,True,105.0,0.826772,90.0,0.708661,0.923604


In summary, HTML extraction did the worst, as it had a high rate of no prediction at all for diagnosis type.
One culprit is that abbreviation / parsing of HTML is not working in spacy.

For Markdown extraction, all approaches did reasonable well
Chunked approaches had a slightly higher rate of null_prediction, but within the ones that were extracted (perhaps easier cases), accuracy was higher
However, using the full text resulted in the highest total *number of matches, suprirsingly using GPT 4o mini model!

## Incorrect examples

For the most promising model (GPT4o mini with basic full text MD extraction), let's look at PMIDS where at least one mismatch occured.


Notes: In many cases a mismatch could still be considered correct because the UMLS code was different but it described the same disease.

Also, sometimes it was impossible to match due to difference in groups detected vs annotated (but still reasonable "correct" annotation)

Finally, often the diagnosis field was subjectively correct, even if not identical (and mismatch on UMLS code).

Perhaps need fully automated human annotation.



IDEA: Add post hoc UMLS codes to spreadsheet



In [11]:
subset = matches[(matches.model_name == 'gpt-4o-2024-05-13') & (matches.chunk == True) & (matches.prompt == 'demographics-zeroshot') & (matches.source == 'md')]
mismatch_pmcids = subset[subset.umls_match == 0].pmcid.unique() # PMICS with at least one mismatch

preds = all_predictions[6] # Get the predictions for the model of interest
preds = preds.sort_values('umls_prob', ascending=False).groupby('group_ix').first().reset_index() # Keep only the highest probability for each prediction group

In [12]:
for pmcid in mismatch_pmcids:
    print(pmcid)
    print("Annotation:")
    display(subset[subset.pmcid == pmcid][['count', 'diagnosis', 'umls']])
    print("Prediction:")
    display(preds[preds.pmcid == pmcid][['count', 'diagnosis', 'umls_cui', 'umls_name', 'umls_prob']])
    

5460048
Annotation:


,count,diagnosis,umls
35,75,chronic dieters,C5687959


Prediction:


,count,diagnosis,umls_cui,umls_name,umls_prob
135,75.0,chronic dieters,C1555457,Chronic,0.657772


6220927
Annotation:


...

,count,diagnosis,umls_cui,umls_name,umls_prob
89,66.0,unilateral hippocampal sclerosis (hippocampal ...,C1504404,Hippocampal Sclerosis,0.907811


4215530
Annotation:


,count,diagnosis,umls
142,24,Very preterm birth\n,C3897192


Prediction:


,count,diagnosis,umls_cui,umls_name,umls_prob
43,24.0,VPT,C4749792,VPTR,0.716879


### Dischordant examples

Excluding HTML and few shot prompts

In [33]:

matches_ex = matches[matches.source != 'html']
matches_ex = matches_ex[matches_ex.prompt != 'demographics-fewshot']

all_predictions_ex = pd.concat(all_predictions)
all_predictions_ex = all_predictions_ex[all_predictions_ex.source != 'html']
all_predictions_ex = all_predictions_ex[all_predictions_ex.prompt != 'demographics-fewshot']

In [34]:
# Groupby pmcid and only show examples where umls_match is not the same for all rows

for pmcid, group in matches_ex.groupby('pmcid'):
    if len(group.umls_match.unique()) > 1:
        display(group[['pmcid', 'umls', 'umls_match', 'umls_prob', 'diagnosis', 'diagnosis_pred', 'source', 'prompt', 'model_name', 'chunk']])

,pmcid,umls,umls_match,umls_prob,diagnosis,diagnosis_pred,source,prompt,model_name,chunk
1,3701149,C2936346,1.0,0.714716,22q11.2 Deletion Syndrome,adolescents with a 22q11.2 Deletion Syndrome,md,demographics-zeroshot,gpt-4o-mini-2024-07-18,True
17,3701149,C0233762,1.0,0.684989,auditory hallucination,adolescents with transient auditory hallucinat...,md,demographics-zeroshot,gpt-4o-mini-2024-07-18,True
1,3701149,C2936346,1.0,0.868713,22q11.2 Deletion Syndrome,22q11.2 Deletion Syndrome,md,demographics-zeroshot,gpt-4o-2024-05-13,False
17,3701149,C0233762,1.0,0.858952,auditory hallucination,transient auditory hallucinations,md,demographics-zeroshot,gpt-4o-2024-05-13,False
1,3701149,C2936346,1.0,0.868713,22q11.2 Deletion Syndrome,22q11.2 Deletion Syndrome,md,demographics-zeroshot-ftstrict,gpt-4o-2024-05-13,False
17,3701149,C0233762,1.0,0.858952,auditory hallucination,transient auditory hallucinations,md,demographics-zeroshot-ftstrict,gpt-4o-2024-05-13,False
1,3701149,C2936346,0.0,NaN,22q11.2 Deletion Syndrome,NaN,md,demographics-zeroshot-ftstrict,gpt-4o-mini-2024-07-18,False
17,3701149,C0233762,1.0,0.935714,auditory hallucination,auditory hallucination (auditory hallucination),md,demographics-zeroshot-ftstrict,gpt-4o-mini-2024-07-18,False
1,3701149,C2936346,1.0,0.868713,22q11.2 Deletion Syndrome,22q11.2 Deletion Syndrome,md,demographics-zeroshot,gpt-4o-mini-2024-07-18,False
17,3701149,C0233762,1.0,1.000000,auditory hallucination,auditory hallucination,md,demographics-zeroshot,gpt-4o-mini-2024-07-18,False


,pmcid,umls,umls_match,umls_prob,diagnosis,diagnosis_pred,source,prompt,model_name,chunk
38,3742334,C4518790,NaN,NaN,chronic marijuana use,NaN,md,demographics-zeroshot,gpt-4o-mini-2024-07-18,True
38,3742334,C4518790,1.0,0.860396,chronic marijuana use,chronic marijuana (marijuana) users,md,demographics-zeroshot,gpt-4o-2024-05-13,False
38,3742334,C4518790,1.0,1.000000,chronic marijuana use,marijuana abuse,md,demographics-zeroshot-ftstrict,gpt-4o-2024-05-13,False
38,3742334,C4518790,1.0,0.860396,chronic marijuana use,chronic marijuana (marijuana) users,md,demographics-zeroshot-ftstrict,gpt-4o-mini-2024-07-18,False
38,3742334,C4518790,NaN,NaN,chronic marijuana use,NaN,md,demographics-zeroshot,gpt-4o-mini-2024-07-18,False
38,3742334,C4518790,1.0,1.000000,chronic marijuana use,marijuana abuse,md,demographics-zeroshot,gpt-4o-2024-05-13,True


,pmcid,umls,umls_match,umls_prob,diagnosis,diagnosis_pred,source,prompt,model_name,chunk
70,4175730,C0409959,1.0,1.0,knee osteoarthritis,chronic osteoarthritis,md,demographics-zeroshot,gpt-4o-mini-2024-07-18,True
70,4175730,C0409959,0.0,NaN,knee osteoarthritis,NaN,md,demographics-zeroshot,gpt-4o-2024-05-13,False
70,4175730,C0409959,0.0,NaN,knee osteoarthritis,NaN,md,demographics-zeroshot-ftstrict,gpt-4o-2024-05-13,False
70,4175730,C0409959,0.0,NaN,knee osteoarthritis,NaN,md,demographics-zeroshot-ftstrict,gpt-4o-mini-2024-07-18,False
70,4175730,C0409959,1.0,1.0,knee osteoarthritis,chronic osteoarthritis,md,demographics-zeroshot,gpt-4o-mini-2024-07-18,False
70,4175730,C0409959,1.0,1.0,knee osteoarthritis,chronic osteoarthritis,md,demographics-zeroshot,gpt-4o-2024-05-13,True


,pmcid,umls,umls_match,umls_prob,diagnosis,diagnosis_pred,source,prompt,model_name,chunk
142,4215530,C3897192,0.0,NaN,Very preterm birth\n,NaN,md,demographics-zeroshot,gpt-4o-mini-2024-07-18,True
142,4215530,C3897192,0.0,NaN,Very preterm birth\n,NaN,md,demographics-zeroshot,gpt-4o-2024-05-13,False
142,4215530,C3897192,0.0,NaN,Very preterm birth\n,NaN,md,demographics-zeroshot-ftstrict,gpt-4o-2024-05-13,False
142,4215530,C3897192,NaN,NaN,Very preterm birth\n,NaN,md,demographics-zeroshot-ftstrict,gpt-4o-mini-2024-07-18,False
142,4215530,C3897192,NaN,NaN,Very preterm birth\n,NaN,md,demographics-zeroshot,gpt-4o-mini-2024-07-18,False
142,4215530,C3897192,0.0,NaN,Very preterm birth\n,NaN,md,demographics-zeroshot,gpt-4o-2024-05-13,True


,pmcid,umls,umls_match,umls_prob,diagnosis,diagnosis_pred,source,prompt,model_name,chunk
36,4473263,C0586325,NaN,NaN,chronic left-hemisphere stroke,NaN,md,demographics-zeroshot,gpt-4o-mini-2024-07-18,True
36,4473263,C0586325,1.0,0.623606,chronic left-hemisphere stroke,left hemisphere stroke,md,demographics-zeroshot,gpt-4o-2024-05-13,False
36,4473263,C0586325,0.0,NaN,chronic left-hemisphere stroke,NaN,md,demographics-zeroshot-ftstrict,gpt-4o-2024-05-13,False
36,4473263,C0586325,1.0,0.544970,chronic left-hemisphere stroke,left hemisphere stroke patients,md,demographics-zeroshot-ftstrict,gpt-4o-mini-2024-07-18,False
36,4473263,C0586325,1.0,0.746261,chronic left-hemisphere stroke,stroke patients,md,demographics-zeroshot,gpt-4o-mini-2024-07-18,False
36,4473263,C0586325,NaN,NaN,chronic left-hemisphere stroke,NaN,md,demographics-zeroshot,gpt-4o-2024-05-13,True


...

,pmcid,umls,umls_match,umls_prob,diagnosis,diagnosis_pred,source,prompt,model_name,chunk
27,9407088,C1510586,1.0,1.000000,autism spectrum disorder,autism spectrum disorder,md,demographics-zeroshot,gpt-4o-mini-2024-07-18,True
28,9407088,C1510586,1.0,1.000000,autism spectrum disorder,autism spectrum disorder,md,demographics-zeroshot,gpt-4o-mini-2024-07-18,True
27,9407088,C1510586,0.0,NaN,autism spectrum disorder,NaN,md,demographics-zeroshot,gpt-4o-2024-05-13,False
28,9407088,C1510586,0.0,NaN,autism spectrum disorder,NaN,md,demographics-zeroshot,gpt-4o-2024-05-13,False
27,9407088,C1510586,1.0,1.000000,autism spectrum disorder,autism spectrum disorder,md,demographics-zeroshot-ftstrict,gpt-4o-2024-05-13,False
28,9407088,C1510586,1.0,1.000000,autism spectrum disorder,autism spectrum disorder,md,demographics-zeroshot-ftstrict,gpt-4o-2024-05-13,False
27,9407088,C1510586,1.0,1.000000,autism spectrum disorder,autism spectrum disorder,md,demographics-zeroshot-ftstrict,gpt-4o-mini-2024-07-18,False
28,9407088,C1510586,1.0,1.000000,autism spectrum disorder,autism spectrum disorder,md,demographics-zeroshot-ftstrict,gpt-4o-mini-2024-07-18,False
27,9407088,C1510586,1.0,0.680409,autism spectrum disorder,autism spectrum disorder patients and normal c...,md,demographics-zeroshot,gpt-4o-mini-2024-07-18,False
28,9407088,C1510586,0.0,NaN,autism spectrum disorder,NaN,md,demographics-zeroshot,gpt-4o-mini-2024-07-18,False


,pmcid,umls,umls_match,umls_prob,diagnosis,diagnosis_pred,source,prompt,model_name,chunk
122,9435010,C0036341,0.0,NaN,schizophrenia,NaN,md,demographics-zeroshot,gpt-4o-mini-2024-07-18,True
122,9435010,C0036341,0.0,NaN,schizophrenia,NaN,md,demographics-zeroshot,gpt-4o-2024-05-13,False
122,9435010,C0036341,0.0,NaN,schizophrenia,NaN,md,demographics-zeroshot-ftstrict,gpt-4o-2024-05-13,False
122,9435010,C0036341,0.0,NaN,schizophrenia,NaN,md,demographics-zeroshot-ftstrict,gpt-4o-mini-2024-07-18,False
122,9435010,C0036341,0.0,NaN,schizophrenia,NaN,md,demographics-zeroshot,gpt-4o-mini-2024-07-18,False
122,9435010,C0036341,NaN,NaN,schizophrenia,NaN,md,demographics-zeroshot,gpt-4o-2024-05-13,True


,pmcid,umls,umls_match,umls_prob,diagnosis,diagnosis_pred,source,prompt,model_name,chunk
32,10870473,C0006012,1.0,1.000000,BPD,borderline personality disorder,md,demographics-zeroshot,gpt-4o-mini-2024-07-18,True
32,10870473,C0006012,1.0,1.000000,BPD,borderline personality disorder,md,demographics-zeroshot,gpt-4o-2024-05-13,False
32,10870473,C0006012,1.0,1.000000,BPD,borderline personality disorder,md,demographics-zeroshot-ftstrict,gpt-4o-2024-05-13,False
32,10870473,C0006012,1.0,0.976984,BPD,Borderline Personality Disorder (borderline pe...,md,demographics-zeroshot-ftstrict,gpt-4o-mini-2024-07-18,False
32,10870473,C0006012,1.0,1.000000,BPD,borderline personality disorder,md,demographics-zeroshot,gpt-4o-mini-2024-07-18,False
32,10870473,C0006012,NaN,NaN,BPD,NaN,md,demographics-zeroshot,gpt-4o-2024-05-13,True


,pmcid,umls,umls_match,umls_prob,diagnosis,diagnosis_pred,source,prompt,model_name,chunk
87,10958407,C2349426,1.0,1.000000,NDPH,new daily persistent headache,md,demographics-zeroshot,gpt-4o-mini-2024-07-18,True
87,10958407,C2349426,1.0,0.948853,NDPH,new daily persistent headache (new daily persi...,md,demographics-zeroshot,gpt-4o-2024-05-13,False
87,10958407,C2349426,1.0,0.948853,NDPH,new daily persistent headache (new daily persi...,md,demographics-zeroshot-ftstrict,gpt-4o-2024-05-13,False
87,10958407,C2349426,1.0,0.948853,NDPH,new daily persistent headache (new daily persi...,md,demographics-zeroshot-ftstrict,gpt-4o-mini-2024-07-18,False
87,10958407,C2349426,1.0,1.000000,NDPH,new daily persistent headache,md,demographics-zeroshot,gpt-4o-mini-2024-07-18,False
87,10958407,C2349426,NaN,NaN,NDPH,NaN,md,demographics-zeroshot,gpt-4o-2024-05-13,True


,pmcid,umls,umls_match,umls_prob,diagnosis,diagnosis_pred,source,prompt,model_name,chunk
4,11024046,C0002395,1.0,0.842912,AD,mild or moderate Alzheimer's disease,md,demographics-zeroshot,gpt-4o-mini-2024-07-18,True
4,11024046,C0002395,1.0,1.000000,AD,Alzheimer's disease,md,demographics-zeroshot,gpt-4o-2024-05-13,False
4,11024046,C0002395,1.0,1.000000,AD,Alzheimer's disease,md,demographics-zeroshot-ftstrict,gpt-4o-2024-05-13,False
4,11024046,C0002395,1.0,1.000000,AD,Alzheimer's disease,md,demographics-zeroshot-ftstrict,gpt-4o-mini-2024-07-18,False
4,11024046,C0002395,1.0,1.000000,AD,Alzheimer's disease,md,demographics-zeroshot,gpt-4o-mini-2024-07-18,False
4,11024046,C0002395,NaN,NaN,AD,NaN,md,demographics-zeroshot,gpt-4o-2024-05-13,True


In [15]:
all_predictions_ex[all_predictions_ex.pmcid == 9435010]

,pmcid,diagnosis,umls_cui,umls_name,umls_prob,count,group_ix,start_char,end_char,chunk,source,prompt,model_name
128,9435010,AVH,C0843593,lavh,0.836862,0.0,410,11452.0,12744.0,True,md,demographics-zeroshot,gpt-4o-mini-2024-07-18
129,9435010,AVH,C0075860,TAVH protocol,0.657447,0.0,410,11452.0,12744.0,True,md,demographics-zeroshot,gpt-4o-mini-2024-07-18
130,9435010,AVH,C0057164,DAVH protocol,0.649644,0.0,410,11452.0,12744.0,True,md,demographics-zeroshot,gpt-4o-mini-2024-07-18
169,9435010,schizophrenia with auditory verbal hallucinati...,C0233762,auditory hallucinations,0.752388,50.0,68,NaN,NaN,False,md,demographics-zeroshot,gpt-4o-2024-05-13
134,9435010,first-episode schizophrenia with auditory verb...,C0233762,auditory hallucinations,0.701497,50.0,69,NaN,NaN,False,md,demographics-zeroshot-ftstrict,gpt-4o-2024-05-13
137,9435010,auditory verbal hallucinations (auditory verba...,C0233762,auditory hallucinations,0.809070,50.0,69,NaN,NaN,False,md,demographics-zeroshot-ftstrict,gpt-4o-mini-2024-07-18
138,9435010,without auditory verbal hallucination (NAVH),C0233762,AUDITORY HALLUCINATION,0.629468,50.0,70,NaN,NaN,False,md,demographics-zeroshot-ftstrict,gpt-4o-mini-2024-07-18
163,9435010,auditory verbal hallucinations (auditory verba...,C0233762,auditory hallucinations,0.809070,50.0,68,NaN,NaN,False,md,demographics-zeroshot,gpt-4o-mini-2024-07-18
164,9435010,without auditory verbal hallucination (NAVH),C0233762,AUDITORY HALLUCINATION,0.629468,50.0,69,NaN,NaN,False,md,demographics-zeroshot,gpt-4o-mini-2024-07-18


In [16]:
# This is an example of chunked approach doing better
all_predictions_ex[all_predictions_ex.pmcid == 5416685]

,pmcid,diagnosis,umls_cui,umls_name,umls_prob,count,group_ix,start_char,end_char,chunk,source,prompt,model_name
80,5416685,unipolar major depression,C0041696,Unipolar Depression,0.824893,20.0,119,15035.0,15652.0,True,md,demographics-zeroshot,gpt-4o-mini-2024-07-18
81,5416685,unipolar major depression,C1269683,major depression,0.803354,20.0,119,15035.0,15652.0,True,md,demographics-zeroshot,gpt-4o-mini-2024-07-18
82,5416685,unipolar major depression,C0005587,depression bipolar,0.658109,20.0,119,15035.0,15652.0,True,md,demographics-zeroshot,gpt-4o-mini-2024-07-18
83,5416685,unipolar major depression,C0041696,Unipolar Depression,0.824893,19.0,120,15035.0,15652.0,True,md,demographics-zeroshot,gpt-4o-mini-2024-07-18
84,5416685,unipolar major depression,C1269683,major depression,0.803354,19.0,120,15035.0,15652.0,True,md,demographics-zeroshot,gpt-4o-mini-2024-07-18
85,5416685,unipolar major depression,C0005587,depression bipolar,0.658109,19.0,120,15035.0,15652.0,True,md,demographics-zeroshot,gpt-4o-mini-2024-07-18
102,5416685,unipolar major depression,C0041696,Unipolar Depression,0.824893,20.0,37,NaN,NaN,False,md,demographics-zeroshot,gpt-4o-2024-05-13
103,5416685,unipolar major depression,C1269683,major depression,0.803354,20.0,37,NaN,NaN,False,md,demographics-zeroshot,gpt-4o-2024-05-13
104,5416685,unipolar major depression,C0005587,depression bipolar,0.658109,20.0,37,NaN,NaN,False,md,demographics-zeroshot,gpt-4o-2024-05-13
105,5416685,unipolar major depression,C0041696,Unipolar Depression,0.824893,19.0,39,NaN,NaN,False,md,demographics-zeroshot,gpt-4o-2024-05-13


## Summary
Sometimes, the chunked approach outperforms the full text approach by chance, and sometimes it does so because it's more likely to use the specific terminology used in the paper
(e.g. Major Depressive Disorded (MDD) vs a more general term (severe depression)). Or a more specific description of the population (e.g. ischemic stroke) vs the topic studies (depression in stroke patients)


However, the chunked approach is more likely to not make a diagnosis prediction overall, and sometimes miss abbreviations.

One additional approach we should try is Abstract and Title + Chunking. This might yield the best overall performance. 
We should also evaluate GPT 4o mini on chunked version.

Or to run full text extraction only when no response is given for diagnosis?